
# MedGemma: Google Model - hosted on Hugging Face
Modified from Notebook: https://github.com/Google-Health/medgemma/blob/main/notebooks/quick_start_with_hugging_face.ipynb



Tutorial: MedGemma using a chest xay stored on your local google drive. Once you can use this simple notebook, play with different models, images, system prompts, and user prompts.

Learn more about the model at the [HAI-DEF developer site](https://developers.google.com/health-ai-developer-foundations/medgemma).

###Note: Make sure you have HF_Token from huggingface so you can access all the models. Put the HF_Token into google secrets on the left - see the key.

In [ ]:
import os
import sys

#google_colab = "google.colab" in sys.modules and not os.environ.get("VERTEX_PRODUCT")

from google.colab import userdata
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")


In [ ]:
#mount drive google colab
from google.colab import drive
drive.mount('/content/drive')

### Install dependencies

In [ ]:
! pip install --upgrade --quiet accelerate bitsandbytes transformers

## Set model variant and configuration
Note you may need to go to the model page on hugging face for google/medgemma-4b-it to get permission to use the model.

In [ ]:
from transformers import BitsAndBytesConfig
import torch

model_id = f"google/medgemma-4b-it"

model_kwargs = dict(
  torch_dtype=torch.bfloat16,
  device_map="auto",
)
model_kwargs["quantization_config"] = BitsAndBytesConfig(load_in_4bit=True)

**Specify image and text inputs**

In [ ]:
import os
from PIL import Image
from IPython.display import Image as IPImage, display, Markdown

**Run model with the `pipeline` API**

In [ ]:
from transformers import pipeline

pipe = pipeline(
    "image-text-to-text",
    model=model_id,
    model_kwargs=model_kwargs,
)

pipe.model.generation_config.do_sample = False

#Core Question/Answer
1. Get image
2. Set up system instructions
3. Create a question- the prompt

In [ ]:
#set image to an image on my google drive

image_path_in_drive = '/content/drive/MyDrive/med-gemma-colab/effusion1.jpeg' # @param {"type":"string"}
image_filename = os.path.basename(image_path_in_drive)
# Load the image using PIL
image = Image.open(image_path_in_drive)


system_instruction = "You are a Senior Emergency Physician Attending." # @param {"type":"string"}
prompt = "Describe this chest xray- patient was in an MVA with chest trauma" # @param {"type":"string"}

messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": system_instruction}]
    },
    {
        "role": "user",
        "content": [
            {"type": "text", "text": prompt},
            {"type": "image", "image": image}
        ]
    }
]



#Find the Answer using your chosen model

In [ ]:
output = pipe(text=messages, max_new_tokens=300)
response = output[0]["generated_text"][-1]["content"]

display(Markdown(f"\n\n---\n\n**[ User ]**\n\n{prompt}"))
display(IPImage(filename=image_path_in_drive, height=300))
display(Markdown(f"\n\n---\n\n**[ MedGemma ]**\n\n{response}\n\n---\n\n"))